In [1]:
import os
from processCapLib import *
from vidRetrieve import *
import logInit  #  order doesn't matter

In [2]:
inpath = '/home/gengshan/public_html/data/Caps_before/'
outpath = '/home/gengshan/public_html/data/Caps_wrt_Tessa/'
logpath = '/home/gengshan/workJul/captionPruning/log/log_'
listOri = [1, 5, 8, 11]   # video to keep unchanged
listRm = [6, 13, 14, 22, 266]  # videos to remove
listMergeSplit_500ms = [12, 29]  # use .5s margin, otherwise dont use margin
pairBegShift = {2: -1.5, 15: -0.5, 16: -0.5, 17: -0.5, 18: -2, 21: -1, 23: -0.5,\
                24: -0.5, 25: -0.5, 27: -1, 54: -1, 129: -0.5, 184: -1, 452: -0.5}  # shift begining time  
pairEndShift = {2: -1.5, 18: -2}  # shift ending time

In [3]:
logger = logInit.makeLogger(logpath)
os.system('cp ' + inpath + '* ' + outpath)  # copy first
os.system('chmod a+w ' + outpath + '*')  # and make it writable for web interface

for it, filename in enumerate(os.listdir(inpath)):
    if it == 800:  # run over all 600 files
        break
        
    if filename[-7:] != '.en.vtt':
        continue  # remove captions in other languages
        
    if title2idx(filename[:-7]) in listOri:
        logger.info('keep ' + filename + ' unchanged...')
        continue
        
    if title2idx(filename[:-7]) in listRm:
        os.system('rm ' + outpath + filename)
        logger.info('rm ' + outpath + filename)
        continue
    
    with open (inpath + filename) as capFile:
        ''' Process a new caption '''
        logger.info(str(it) + '[#' + vid2Num(filename.split('.')[0]) + ']: ' + inpath + filename + '\n')
        caps = capFile.readlines()
        
        ''' create dictionary '''
        capPairs = creatDict(caps)
        
        ''' remove speaker names and special patterns '''
        capPairs = rmNameBefColon(capPairs)
        capPairs = rmPattern(capPairs)
        
        ''' clean empty subtitles '''
        capPairs = filterByLength(capPairs)
        
        ''' merge broken sentences to the previous ones. Also clear empty caps '''
        capPairs = merge2sents(capPairs)
    
        ''' split text on sentence '''
        capPairs = split2sents(capPairs)
        
        ''' filter short clips '''
        capPairs = filterByTime(capPairs, 0.5)
        
        ''' add margins '''
        if title2idx(filename[:-7]) in listMergeSplit_500ms:
            capPairs = addMargin(capPairs, 0.5)
        
        ''' shift time stamps '''
        if title2idx(filename[:-7]) in pairBegShift.keys():
            capPairs = shiftBeg(capPairs, pairBegShift[title2idx(filename[:-7])])
        if title2idx(filename[:-7]) in pairEndShift.keys():
            capPairs = shiftEnd(capPairs, pairEndShift[title2idx(filename[:-7])])
        
        ''' dump to another path '''
        dumpVTT(outpath + filename, capPairs)

logger.handlers[0].close()

{'stream': <open file '/home/gengshan/workJul/captionPruning/log/log_08_05_12_53', mode 'a' at 0x7f0bc4423a50>, 'encoding': None, 'lock': <_RLock owner=None count=0>, 'level': 10, '_name': None, 'delay': 0, 'baseFilename': '/home/gengshan/workJul/captionPruning/log/log_08_05_12_53', 'mode': 'a', 'filters': [], 'formatter': <logging.Formatter object at 0x7f0b5c6b1bd0>}
